### Valores ausentes

A menudo, las fuentes de datos están incompletas, lo que significa que le faltarán datos, tiene 3 opciones básicas para completar los datos faltantes. Usted personalmente tendrá que tomar la decisión de cuál es el enfoque correcto dependiendo de su situación:

* Mantener los valores faltantes
* Eliminar los registros/columnas con valores faltantes
* Imputar con algún otro valor

In [0]:
generation = "mod4gen13"

In [0]:
df = spark.read.csv(f"/mnt/{generation}/input/contains_null.csv", inferSchema=True, header=True)

In [0]:
df.display()

#### Eliminación de valores ausentes

Es posible usar el método `.na` para trabajar con datos faltantes. El comando `drop()` tiene los siguientes parámetros:

    df.na.drop(how='any', thresh=None, subset=None)
    
    * param how: 'any' or 'all'.
    
        If 'any', drop a row if it contains any nulls.
        If 'all', drop a row only if all its values are null.
    
    * param thresh: int, default None
    
        If specified, drop rows that have less than `thresh` non-null values.
        This overwrites the `how` parameter.
        
    * param subset: 
        optional list of column names to consider.

In [0]:
# Tirar cualquier fila que contenga datos faltantes
df.na.drop().display()

In [0]:
df.na.drop(how='any').display()

In [0]:
# Retiene el registro si se cumple la condición de contener menos de 2 valores faltantes
df.na.drop(thresh=2).display()

In [0]:
# Elimina valores faltantes para una variable específica 
df.na.drop(subset=["Sales"]).display()

In [0]:
df.na.drop(how='all').display()

In [0]:
df.drop("Name").display()

#### Imputación

También podemos imputar nuevos valores. Si se cuenta con valores nulos en variable con distintos tipos de datos, Spark es lo suficientemente "inteligente" como para hacer coincidir los tipos de datos.

In [0]:
df.na.fill('MISSING').display()

In [0]:
df.na.fill(0).display()

In [0]:
# Sin embargo, se recomienda especificar las columnas que se desean imputar junto con su respectivo valor
df.na.fill('No Name', subset=['Name']).display()

In [0]:
from pyspark.sql.functions import mean

In [0]:
mean_val = df.select(mean(df["Sales"])).collect()
mean_val[0][0]

In [0]:
mean_sales = mean_val[0][0]

In [0]:
df.na.fill(mean_sales, subset=["Sales"]).display()

In [0]:
# Misma operación de imputación pero en una solo línea de código
df.na.fill(df.select(mean(df["Sales"])).collect()[0][0], ["Sales"]).display()